In [1]:
from qnet.log_init import initialize_logger
initialize_logger()


In [2]:
from qnet.study import OptunaStudy
from qnet import ROOT
import yaml
import os
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv(os.path.join(ROOT, "data/kin8nm.csv"))

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

base_config_path = os.path.join(ROOT, "experiments/base_config.yaml")

base_config = yaml.full_load(open(base_config_path))
model_config = {
    "categorical_features":[],
    "continuous_features":df.columns.to_list()[:-1],
    "multi_label_categorical_features":[],
    "target":df.columns.to_list()[-1:],
    "yeo_transform":False
}

save_dir = os.path.join(ROOT, "experiments/kin8nm/study")
study = OptunaStudy(
    df=train_df,
    model_config=model_config,
    base_config=base_config,
    save_folder=save_dir,
    n_trials=50,
    optimize_model_architecture=False,
    optimize_batch_size=True,
    optimize_momentum=True,
    optimize_mse=True,
)

#study.run_study()

In [3]:
from qnet.train import Engine

train_config = yaml.full_load(open(r"/Users/jackarmand/Documents/GitHub/quantile-net/qnet/experiments/concrete/study/train_config_02_02_11_55.yaml"))
engine = Engine(
    train_config,
    model_config,
    df_init=train_df
)
engine.train()
dir = engine.save(os.path.join(ROOT, "experiments/kin8nm/model"), append_auto_dir=False)
engine = Engine.load(dir)

[2022-02-02 14:18:34,143    INFO], [models.py:38 - __init__()], Model on device: cpu


Epoch    56: reducing learning rate of group 0 to 8.1255e-04.
Epoch    67: reducing learning rate of group 0 to 1.6251e-04.
Epoch   183: reducing learning rate of group 0 to 3.2502e-05.
Epoch   206: reducing learning rate of group 0 to 6.5004e-06.
Epoch   214: reducing learning rate of group 0 to 1.3001e-06.


[2022-02-02 14:22:32,816    INFO], [train.py:312 - train()], Model training stopped after 236 epochs.
[2022-02-02 14:22:32,816    INFO], [train.py:313 - train()], Best testing loss: -23.9296
[2022-02-02 14:22:32,850    INFO], [models.py:38 - __init__()], Model on device: None
[2022-02-02 14:22:32,858    INFO], [models.py:176 - load()], Loading device : cpu


Epoch   222: reducing learning rate of group 0 to 2.6002e-07.


In [4]:
from qnet.predictions import Predictions

dir = os.path.join(ROOT, "experiments/kin8nm/model")
pred_maker = Predictions(dir)


[2022-02-02 14:22:51,723    INFO], [models.py:38 - __init__()], Model on device: None


In [5]:
import numpy as np
from qnet.probability import generate_cdf_function

log_list = []
for row in test_df.index:
    qn_predictions = pred_maker.predict(pd.DataFrame(test_df.loc[row]).T)
    spline, xmin, xmax = generate_cdf_function(pred_maker.model.config.quantiles, qn_predictions[:-1], la=1)
    d_spline = spline.derivative()
    likelihood = d_spline(test_df.loc[row][model_config["target"]])
    if likelihood<1e-15:
        likelihood = 1e-15
        print("rats")
    log_list.append(-np.log(likelihood))

np.mean(log_list)

rats


/Users/jackarmand/Documents/GitHub/quantile-net/venv/lib/python3.9/site-packages/numpy/core/_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


array([-1.24428446])

In [ ]:
import plotly.graph_objs as go



fig = go.Figure()
fig.add_trace()